In [1]:
%run moon_code

In [2]:
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

In [3]:
kospi200, name_dict, code_dict = load_code_name_data("./data/data.csv", True)
party_df = load_party_data('party_df.csv')

In [102]:
with open('ks11_20181008.pickle', 'rb') as handle:
    ks11 = pickle.load(handle)

In [55]:
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)

In [486]:
raw_data['AK홀딩스'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,5726.419922,5796.470215,5673.890137,5778.959961,5778.959961,33132
2010-01-05,5743.939941,5849.009766,5708.910156,5708.910156,5708.910156,67542
2010-01-06,5708.910156,5743.939941,5638.859863,5708.910156,5708.910156,72348
2010-01-07,5708.910156,5778.959961,5621.350098,5761.450195,5761.450195,84399
2010-01-08,5778.959961,5813.979980,5708.910156,5796.470215,5796.470215,22847


In [511]:
panel_ = {}
for stock, df in raw_data.items():
    panel_[stock] = df['Close'].copy()
    panel_[stock].name = stock

In [488]:
q = pd.Series(index = panel_['삼성전자'].index)
for stock, df in panel_.items():
    q = pd.concat((q, df),axis=1)
del q[0]

In [489]:
p = p.unstack()

In [490]:
q = q.unstack()

In [491]:
panel = pd.concat((p, q), axis=1)
panel.columns = ['Open', 'Close']

In [492]:
res = {}
for col in raw_data['삼성전자'].columns:
    panel_ = {}
    p = pd.Series(index = raw_data['삼성전자'].index)
    for stock, df in raw_data.items():
        a = df[col].copy()
        a.name = stock
        p = pd.concat((p, a),axis=1)
    del p[0]
    res[col] = p

In [493]:
panel = pd.concat((res['Open'].unstack(),
                   res['High'].unstack(),
                   res['Low'].unstack(),
                   res['Close'].unstack(),
                   res['Adj Close'].unstack(),
                   res['Volume'].unstack()),
                  axis=1)
panel.columns = raw_data['삼성전자'].columns

In [513]:
panel.head()

Open         High          Low        Close  \
      Date                                                             
AK홀딩스 2010-01-04  5726.419922  5796.470215  5673.890137  5778.959961   
      2010-01-05  5743.939941  5849.009766  5708.910156  5708.910156   
      2010-01-06  5708.910156  5743.939941  5638.859863  5708.910156   
      2010-01-07  5708.910156  5778.959961  5621.350098  5761.450195   
      2010-01-08  5778.959961  5813.979980  5708.910156  5796.470215   

                    Adj Close   Volume  
      Date                              
AK홀딩스 2010-01-04  5778.959961  33132.0  
      2010-01-05  5708.910156  67542.0  
      2010-01-06  5708.910156  72348.0  
      2010-01-07  5761.450195  84399.0  
      2010-01-08  5796.470215  22847.0

In [105]:
# KOSPI 수익률 추가
returns_day = [1, 3, 5, 7, 10, 12, 14, 16]
for i in returns_day:
    ks11['returnsClosePrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Close.shift(i))
    ks11['returnsOpenPrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Open.shift(i))
kospi_index_del = ks11[ks11.Volume == 0][ks11.columns[:6]].index

In [145]:
pan = panel.unstack().fillna(0).stack()

In [135]:
for stock in pan.index.get_level_values(0).unique():
    pan.loc[stock] = pan.loc[stock].drop(kospi_index_del)

In [159]:
len(kospi_index_del) * 202

2424

In [160]:
len(pan) - 2424

433492

In [161]:
pan = pan.swaplevel().sortlevel(0).drop(kospi_index_del).swaplevel().sortlevel(0)

In [169]:
(pan == 0).sum()

Open         33888
High         33888
Low          33888
Close        33888
Adj Close    33888
Volume       35458
dtype: int64

In [186]:
pan = pan[~((pan == 0).any(1))]

In [189]:
(pan == 0).sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [192]:
pan.head()

Open         High          Low        Close  \
      Date                                                             
AK홀딩스 2010-01-04  5726.419922  5796.470215  5673.890137  5778.959961   
      2010-01-05  5743.939941  5849.009766  5708.910156  5708.910156   
      2010-01-06  5708.910156  5743.939941  5638.859863  5708.910156   
      2010-01-07  5708.910156  5778.959961  5621.350098  5761.450195   
      2010-01-08  5778.959961  5813.979980  5708.910156  5796.470215   

                    Adj Close   Volume  
      Date                              
AK홀딩스 2010-01-04  5778.959961  33132.0  
      2010-01-05  5708.910156  67542.0  
      2010-01-06  5708.910156  72348.0  
      2010-01-07  5761.450195  84399.0  
      2010-01-08  5796.470215  22847.0

In [215]:
pan.index.names = ['Stock', 'Date']

In [228]:
for stock, data in pan.groupby('Stock'):
    df = data.copy()
    df["pct_change"] = df.Close.pct_change()

In [245]:
np.tile(ks11['returnsClosePrevRaw{}'.format(5)].values, 5)

10755

In [278]:
import time
start = time.time()
pan['pct_change'] = (pan.groupby('Stock').apply(lambda x : x['Close'].pct_change())).values
for i in range(3, 21, 2):
    pan['close_ma_{}'.format(i)] = (pan.groupby('Stock').apply(lambda x : x['Close'].rolling(window=i).mean())).values
    pan['volume_ma_{}'.format(i)] = (pan.groupby('Stock').apply(lambda x : x['Volume'].rolling(window=i).mean())).values
for i in [7,21,30,60,90,100,120]:
    pan["his_vol_{}".format(i)] = (pan.groupby('Stock').apply(lambda x : x["pct_change"].rolling(window = i).std()*(252**0.5))).values
for i in returns_day:
    pan['returnsClosePrevRaw{}'.format(i)]= (pan.groupby('Stock').apply(lambda x : np.log(x['Open'] / x['Close'].shift(i)))).values
    pan['returnsOpenPrevRaw{}'.format(i)] = (pan.groupby('Stock').apply(lambda x : np.log(x['Open'] / x['Open'].shift(i)))).values
for i in [1,3,5,7,10]:
    pan['Y3_{}'.format(i)] = (pan.groupby('Stock').apply(lambda x : pd.Series(np.where(
            x['Close'].rolling(5).mean() - x['Close'].shift(1).rolling(5).mean() >= 0, 1, 0)))).values
    for shift in [1,2,3,4,5]:
        pan['sh{}_Y3_{}'.format(shift, i)] = (pan.groupby('Stock').apply(lambda x : x['Y3_{}'.format(i)].shift(-1*shift))).values
elapse = time.time() -start

In [279]:
print(elapse)

49.981932163238525


# 새롭게

In [477]:
panel = pd.Panel(raw_data)

In [478]:
panel

<class 'pandas.core.panel.Panel'>
Dimensions: 202 (items) x 2158 (major_axis) x 6 (minor_axis)
Items axis: AK홀딩스 to 휴켐스
Major_axis axis: 2010-01-04 00:00:00 to 2018-10-01 00:00:00
Minor_axis axis: Open to Volume

In [485]:
panel.to_frame(False)

AK홀딩스       BGF    BGF리테일    BNK금융지주        CJ  \
Date       minor                                                              
2010-01-04 Open        5726.419922       NaN       NaN        NaN   62000.0   
           High        5796.470215       NaN       NaN        NaN   62200.0   
           Low         5673.890137       NaN       NaN        NaN   61100.0   
           Close       5778.959961       NaN       NaN        NaN   62200.0   
           Adj Close   5778.959961       NaN       NaN        NaN   62200.0   
           Volume     33132.000000       NaN       NaN        NaN  130619.0   
2010-01-05 Open        5743.939941       NaN       NaN        NaN   62200.0   
           High        5849.009766       NaN       NaN        NaN   63700.0   
           Low         5708.910156       NaN       NaN        NaN   61500.0   
           Close       5708.910156       NaN       NaN        NaN   63200.0   
           Adj Close   5708.910156       NaN       NaN        NaN   63200.0   
           Volume     67542.000000       NaN       NaN        NaN  290082.0   
2010-01-06 Open        5708.910156       NaN       NaN        NaN   63000.0   
           High        5743.939941       NaN       NaN        NaN   64700.0   
           Low         5638.859863       NaN       NaN        NaN   62600.0   
           Close       5708.910156       NaN       NaN        NaN   63200.0   
           Adj Close   5708.910156       NaN       NaN        NaN   63200.0   
           Volume     72348.000000       NaN       NaN        NaN  163328.0   
2010-01-07 Open        5708.910156       NaN       NaN        NaN   64200.0   
           High        5778.959961       NaN       NaN        NaN   65200.0   
           Low         5621.350098       NaN       NaN        NaN   63200.0   
           Close       5761.450195       NaN       NaN        NaN   64100.0   
           Adj Close   5761.450195       NaN       NaN        NaN   64100.0   
           Volume     84399.000000       NaN       NaN        NaN  259814.0   
2010-01-08 Open        5778.959961       NaN       NaN        NaN   64300.0   
           High        5813.979980       NaN       NaN        NaN   67500.0   
           Low         5708.910156       NaN       NaN        NaN   64200.0   
           Close       5796.470215       NaN       NaN        NaN   67400.0   
           Adj Close   5796.470215       NaN       NaN        NaN   67400.0   
           Volume     22847.000000       NaN       NaN        NaN  237033.0   
...                            ...       ...       ...        ...       ...   
2018-09-20 Open       62300.000000    9230.0  195500.0     8190.0  132500.0   
           High       62300.000000    9260.0  200000.0     8260.0  134500.0   
           Low        60500.000000    9160.0  194500.0     8160.0  132500.0   
           Close      60600.000000    9170.0  195000.0     8240.0  133500.0   
           Adj Close  60600.000000    9170.0  195000.0     8240.0  133500.0   
           Volume     28799.000000  211485.0   21026.0   731025.0   43767.0   
2018-09-21 Open       61400.000000    9240.0  200500.0     8260.0  133500.0   
           High       61800.000000    9370.0  201000.0     8370.0  136500.0   
           Low        59000.000000    9170.0  195000.0     8250.0  133000.0   
           Close      60300.000000    9370.0  201000.0     8340.0  135500.0   
           Adj Close  60300.000000    9370.0  201000.0     8340.0  135500.0   
           Volume     60168.000000  204811.0   23586.0  1007624.0   79972.0   
2018-09-27 Open       60000.000000    9360.0  201000.0     8310.0  136000.0   
           High       62400.000000    9420.0  215000.0     8510.0  136500.0   
           Low        60000.000000    9250.0  194500.0     8300.0  133500.0   
           Close      62000.000000    9350.0  212500.0     8500.0  134000.0   
           Adj Close  62000.000000    9350.0  212500.0     8500.0  134000.0   
           Volume     36085.000000  222638.0   50760.0   956726.0   95455.0   


In [455]:
len(panel.loc['Close'])

2158

In [456]:
def get_multi_index(col_name):
    return pd.MultiIndex.from_arrays([[col_name]*len(panel.loc['Close']),
                                      panel.index.get_level_values(1).unique().tolist()])

In [457]:
index = get_multi_index('pct_change')


In [464]:
panel

AK홀딩스       BGF   BGF리테일    BNK금융지주        CJ  \
minor     Date                                                               
Adj Close 2010-01-04   5778.959961       NaN      NaN        NaN   62200.0   
          2010-01-05   5708.910156       NaN      NaN        NaN   63200.0   
          2010-01-06   5708.910156       NaN      NaN        NaN   63200.0   
          2010-01-07   5761.450195       NaN      NaN        NaN   64100.0   
          2010-01-08   5796.470215       NaN      NaN        NaN   67400.0   
          2010-01-11   5813.979980       NaN      NaN        NaN   68200.0   
          2010-01-12   5831.500000       NaN      NaN        NaN   67600.0   
          2010-01-13   5743.939941       NaN      NaN        NaN   65200.0   
          2010-01-14   5673.890137       NaN      NaN        NaN   68000.0   
          2010-01-15   5849.009766       NaN      NaN        NaN   71100.0   
          2010-01-18   5971.589844       NaN      NaN        NaN   71000.0   
          2010-01-19   5936.569824       NaN      NaN        NaN   70500.0   
          2010-01-20   5919.060059       NaN      NaN        NaN   72700.0   
          2010-01-21   5831.500000       NaN      NaN        NaN   72700.0   
          2010-01-22   5813.979980       NaN      NaN        NaN   70000.0   
          2010-01-25   5743.939941       NaN      NaN        NaN   67700.0   
          2010-01-26   5673.890137       NaN      NaN        NaN   66700.0   
          2010-01-27   5673.890137       NaN      NaN        NaN   67600.0   
          2010-01-28   5726.419922       NaN      NaN        NaN   68900.0   
          2010-01-29   5568.819824       NaN      NaN        NaN   67200.0   
          2010-02-01   5603.839844       NaN      NaN        NaN   65400.0   
          2010-02-02   5603.839844       NaN      NaN        NaN   65400.0   
          2010-02-03   5568.819824       NaN      NaN        NaN   66800.0   
          2010-02-04   5603.839844       NaN      NaN        NaN   65500.0   
          2010-02-05   5603.839844       NaN      NaN        NaN   64000.0   
          2010-02-08   5568.819824       NaN      NaN        NaN   63200.0   
          2010-02-09   5428.720215       NaN      NaN        NaN   63800.0   
          2010-02-10   5376.180176       NaN      NaN        NaN   63900.0   
          2010-02-11   5376.180176       NaN      NaN        NaN   64500.0   
          2010-02-12   5271.109863       NaN      NaN        NaN   64700.0   
...                            ...       ...      ...        ...       ...   
Volume    2018-08-16  19597.000000  179546.0  20254.0  1522433.0   66262.0   
          2018-08-17  17951.000000  122116.0  12424.0  1618414.0  100717.0   
          2018-08-20  22586.000000  149118.0  11994.0   749990.0   53507.0   
          2018-08-21  30470.000000  139141.0  21098.0   555311.0   40399.0   
          2018-08-22  51412.000000  124107.0  31235.0   356554.0   34023.0   
          2018-08-23  42849.000000  245474.0  14234.0   593428.0   95824.0   
          2018-08-24  32965.000000  133146.0  21653.0   400867.0  118459.0   
          2018-08-27  35005.000000  225168.0  26875.0   590677.0   67489.0   
          2018-08-28  40689.000000  509298.0  58210.0   770237.0   66626.0   
          2018-08-29  19633.000000  325142.0  13795.0   521971.0   64548.0   
          2018-08-30  24818.000000  309588.0  16754.0   662141.0  141496.0   
          2018-08-31  24060.000000  186172.0  23304.0  1105547.0  100410.0   
          2018-09-03  11073.000000  229515.0  18743.0   420797.0   37159.0   
          2018-09-04  26777.000000  183923.0  12813.0   334147.0   35215.0   
          2018-09-05  17956.000000  136941.0  14888.0   602160.0   39148.0   
          2018-09-06  30220.000000  167402.0  13831.0   463045.0   44645.0   
          2018-09-07  28948.000000  257328.0  26900.0   599353.0   51124.0   
          2018-09-10  36607.000000  743658.0  52077.0   618614.0   45055.0   
          2018-09-11  38849.000000  213120.0  34436

In [419]:
panel.loc['Close'].pct_change()

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대중공업,현대중공업지주,현대차,현대해상,현대홈쇼핑,호텔신라,효성,효성중공업,후성,휴켐스
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.012122,NaN,NaN,NaN,0.016077,0.004739,-0.001835,0.062802,-0.017544,0.006667,...,-0.005811,NaN,-0.075630,-0.007712,NaN,0.015945,0.035169,NaN,-0.006570,-0.032038
2010-01-06,0.000000,NaN,NaN,NaN,0.000000,0.018868,0.036765,0.002273,0.055060,-0.018543,...,0.038014,NaN,0.009091,0.046632,NaN,-0.004484,0.049829,NaN,0.014550,0.017419
2010-01-07,0.009203,NaN,NaN,NaN,0.014241,0.006944,0.003546,-0.011338,0.005642,0.006748,...,0.092957,NaN,-0.045045,0.000000,NaN,-0.029279,-0.031284,NaN,0.007823,-0.017121
2010-01-08,0.006078,NaN,NaN,NaN,0.051482,0.048276,0.017668,0.000000,-0.009818,0.006702,...,0.056696,NaN,0.000000,-0.009901,NaN,-0.009281,0.035635,NaN,0.005175,0.006967
2010-01-11,0.003021,NaN,NaN,NaN,0.011869,-0.008772,0.031250,-0.009174,0.007082,-0.015979,...,0.004883,NaN,-0.042453,-0.015000,NaN,0.046838,0.002150,NaN,0.042471,-0.024217
2010-01-12,0.003013,NaN,NaN,NaN,-0.008798,0.044248,-0.016835,0.009259,0.015471,-0.039242,...,-0.009711,NaN,0.014778,0.025381,NaN,-0.026846,-0.004291,NaN,-0.011111,0.015957
2010-01-13,-0.015015,NaN,NaN,NaN,-0.035503,-0.012712,-0.029110,-0.032110,0.013850,-0.015493,...,-0.056370,NaN,-0.009709,0.022277,NaN,-0.002299,-0.030171,NaN,0.009988,-0.003492
2010-01-14,-0.012195,NaN,NaN,NaN,0.042945,-0.002146,0.021164,0.037915,0.035519,0.008584,...,0.059738,NaN,0.014706,0.002421,NaN,0.041475,0.018888,NaN,-0.013597,-0.003504


# 다시 새롭게

In [465]:
pd.DataFrame(raw_data)

ValueError: If using all scalar values, you must pass an index

# 쭈밤...

In [514]:
t = pd.DataFrame(columns = pd.MultiIndex.from_arrays([[stock]*6, df.columns]))
t

Empty DataFrame
Columns: [(휴켐스, Open), (휴켐스, High), (휴켐스, Low), (휴켐스, Close), (휴켐스, Adj Close), (휴켐스, Volume)]
Index: []

In [516]:
data = {}
for stock, df in raw_data.items():
    dff = df.copy()
    dff.columns = pd.MultiIndex.from_arrays([[stock]*6, df.columns])
    data[stock] = dff

In [565]:
p = pd.DataFrame()
for stock, df in data.items():
    p = pd.concat((p, df), axis=1)

In [566]:
p.head()

AK홀딩스                                                      \
                   Open         High          Low        Close    Adj Close   
Date                                                                          
2010-01-04  5726.419922  5796.470215  5673.890137  5778.959961  5778.959961   
2010-01-05  5743.939941  5849.009766  5708.910156  5708.910156  5708.910156   
2010-01-06  5708.910156  5743.939941  5638.859863  5708.910156  5708.910156   
2010-01-07  5708.910156  5778.959961  5621.350098  5761.450195  5761.450195   
2010-01-08  5778.959961  5813.979980  5708.910156  5796.470215  5796.470215   

                   BGF                  ...        후성                    \
           Volume Open High Low Close   ...       Low   Close Adj Close   
Date                                    ...                               
2010-01-04  33132  NaN  NaN NaN   NaN   ...    3800.0  3805.0    3805.0   
2010-01-05  67542  NaN  NaN NaN   NaN   ...    3760.0  3780.0    3780.0   
2010-01-06  72348  NaN  NaN NaN   NaN   ...    3770.0  3835.0    3835.0   
2010-01-07  84399  NaN  NaN NaN   NaN   ...    3835.0  3865.0    3865.0   
2010-01-08  22847  NaN  NaN NaN   NaN   ...    3840.0  3885.0    3885.0   

                              휴켐스                                            \
             Volume          Open          High           Low         Close   
Date                                                                          
2010-01-04   784870  18110.599609  18261.300781  17688.699219  17869.500000   
2010-01-05  1010749  17869.500000  17869.500000  17236.699219  17297.000000   
2010-01-06  1064106  17417.500000  17718.900391  17236.699219  17598.300781   
2010-01-07  2206727  17628.500000  17839.400391  17297.000000  17297.000000   
2010-01-08   996558  17297.000000  17538.099609  17297.000000  17417.500000   

                                  
               Adj Close  Volume  
Date                              
2010-01-04  17869.500000  344720  
2010-01-05  17297.000000  671702  
2010-01-06  17598.300781  276467  
2010-01-07  17297.000000  232912  
2010-01-08  17417.500000  181563  

[5 rows x 1212 columns]

In [573]:
q = p.T.swaplevel().sortlevel().T

In [606]:
def get_multi_index(col_name):
    return pd.MultiIndex.from_product([[col_name],list(raw_data.keys())])

In [607]:
ind = get_multi_index('pct_change')

In [583]:
test = q.Close.pct_change().copy()
test.columns = ind

In [584]:
pd.concat((q, test), axis=1)

Adj Close                                                    \
                   AK홀딩스     BGF    BGF리테일 BNK금융지주        CJ        CJ CGV   
Date                                                                         
2010-01-04   5778.959961     NaN       NaN     NaN   62200.0  19882.162109   
2010-01-05   5708.910156     NaN       NaN     NaN   63200.0  19976.390625   
2010-01-06   5708.910156     NaN       NaN     NaN   63200.0  20353.304688   
2010-01-07   5761.450195     NaN       NaN     NaN   64100.0  20494.646484   
2010-01-08   5796.470215     NaN       NaN     NaN   67400.0  21484.044922   
2010-01-11   5813.979980     NaN       NaN     NaN   68200.0  21295.587891   
2010-01-12   5831.500000     NaN       NaN     NaN   67600.0  22237.871094   
2010-01-13   5743.939941     NaN       NaN     NaN   65200.0  21955.183594   
2010-01-14   5673.890137     NaN       NaN     NaN   68000.0  21908.070313   
2010-01-15   5849.009766     NaN       NaN     NaN   71100.0  21295.587891   
2010-01-18   5971.589844     NaN       NaN     NaN   71000.0  21578.271484   
2010-01-19   5936.569824     NaN       NaN     NaN   70500.0  21295.587891   
2010-01-20   5919.060059     NaN       NaN     NaN   72700.0  21342.699219   
2010-01-21   5831.500000     NaN       NaN     NaN   72700.0  20730.216797   
2010-01-22   5813.979980     NaN       NaN     NaN   70000.0  20824.447266   
2010-01-25   5743.939941     NaN       NaN     NaN   67700.0  21766.730469   
2010-01-26   5673.890137     NaN       NaN     NaN   66700.0  21672.500000   
2010-01-27   5673.890137     NaN       NaN     NaN   67600.0  19976.390625   
2010-01-28   5726.419922     NaN       NaN     NaN   68900.0  20306.189453   
2010-01-29   5568.819824     NaN       NaN     NaN   67200.0  20259.076172   
2010-02-01   5603.839844     NaN       NaN     NaN   65400.0  19646.591797   
2010-02-02   5603.839844     NaN       NaN     NaN   65400.0  20164.847656   
2010-02-03   5568.819824     NaN       NaN     NaN   66800.0  20730.216797   
2010-02-04   5603.839844     NaN       NaN     NaN   65500.0  21201.355469   
2010-02-05   5603.839844     NaN       NaN     NaN   64000.0  20588.875000   
2010-02-08   5568.819824     NaN       NaN     NaN   63200.0  20730.216797   
2010-02-09   5428.720215     NaN       NaN     NaN   63800.0  21295.587891   
2010-02-10   5376.180176     NaN       NaN     NaN   63900.0  21201.355469   
2010-02-11   5376.180176     NaN       NaN     NaN   64500.0  21578.271484   
2010-02-12   5271.109863     NaN       NaN     NaN   64700.0  21389.814453   
...                  ...     ...       ...     ...       ...           ...   
2018-08-16  72200.000000  9380.0  161000.0  8250.0  141000.0  53800.000000   
2018-08-17  71700.000000  9410.0  161500.0  8100.0  138500.0  52000.000000   
2018-08-20  71000.000000  9430.0  165500.0  8220.0  137500.0  51100.000000   
2018-08-21  70500.000000  9540.0  165000.0  8280.0  140000.0  52200.000000   
2018-08-22  69100.000000  9570.0  161000.0  8260.0  139000.0  52800.000000   
2018-08-23  67900.000000  9400.0  159000.0  8200.0  136000.0  53000.000000   
2018-08-24  67200.000000  9370.0  159000.0  8220.0  134500.0  52500.000000   
2018-08-27  67200.000000  9520.0  166500.0  8200.0  135000.0  55400.000000   
2018-08-28  67400.000000  9830.0  180000.0  8300.0  135000.0  54600.000000   
2018-08-29  67000.000000  9670.0  176500.0  8290.0  135500.0  54100.000000   
2018-08-30  66000.000000  9520.0  170000.0  8250.0  132500.0  53500.000000   
2018-08-31  67200.000000  9440.0  171500.0  8340.0  132000.0  53000.000000   
2018-09-03  66300.000000  9320.0  175500.0  8310.0  132000.0  52500.000000   
2018-09-04  65300.000000  9230.0  174500.0  8270.0  133500.0  52900.000000   
2018-09-05  66000.000000  9210.0  176000.0  8210.0  132500.0  52900.000000   
2018-09-06  65000.000000  9160.0  172500.0  8170.0  133500.0  52400.000000   
2018-09-07  63500.000000  9320.0  181500.0  8320.0  133500.0  52000.000000   
2018-09-10  61700.000000  9660.0  191

In [564]:
p

Adj Close                                                    \
                   AK홀딩스     BGF    BGF리테일 BNK금융지주        CJ        CJ CGV   
Date                                                                         
2010-01-04   5778.959961     NaN       NaN     NaN   62200.0  19882.162109   
2010-01-05   5708.910156     NaN       NaN     NaN   63200.0  19976.390625   
2010-01-06   5708.910156     NaN       NaN     NaN   63200.0  20353.304688   
2010-01-07   5761.450195     NaN       NaN     NaN   64100.0  20494.646484   
2010-01-08   5796.470215     NaN       NaN     NaN   67400.0  21484.044922   
2010-01-11   5813.979980     NaN       NaN     NaN   68200.0  21295.587891   
2010-01-12   5831.500000     NaN       NaN     NaN   67600.0  22237.871094   
2010-01-13   5743.939941     NaN       NaN     NaN   65200.0  21955.183594   
2010-01-14   5673.890137     NaN       NaN     NaN   68000.0  21908.070313   
2010-01-15   5849.009766     NaN       NaN     NaN   71100.0  21295.587891   
2010-01-18   5971.589844     NaN       NaN     NaN   71000.0  21578.271484   
2010-01-19   5936.569824     NaN       NaN     NaN   70500.0  21295.587891   
2010-01-20   5919.060059     NaN       NaN     NaN   72700.0  21342.699219   
2010-01-21   5831.500000     NaN       NaN     NaN   72700.0  20730.216797   
2010-01-22   5813.979980     NaN       NaN     NaN   70000.0  20824.447266   
2010-01-25   5743.939941     NaN       NaN     NaN   67700.0  21766.730469   
2010-01-26   5673.890137     NaN       NaN     NaN   66700.0  21672.500000   
2010-01-27   5673.890137     NaN       NaN     NaN   67600.0  19976.390625   
2010-01-28   5726.419922     NaN       NaN     NaN   68900.0  20306.189453   
2010-01-29   5568.819824     NaN       NaN     NaN   67200.0  20259.076172   
2010-02-01   5603.839844     NaN       NaN     NaN   65400.0  19646.591797   
2010-02-02   5603.839844     NaN       NaN     NaN   65400.0  20164.847656   
2010-02-03   5568.819824     NaN       NaN     NaN   66800.0  20730.216797   
2010-02-04   5603.839844     NaN       NaN     NaN   65500.0  21201.355469   
2010-02-05   5603.839844     NaN       NaN     NaN   64000.0  20588.875000   
2010-02-08   5568.819824     NaN       NaN     NaN   63200.0  20730.216797   
2010-02-09   5428.720215     NaN       NaN     NaN   63800.0  21295.587891   
2010-02-10   5376.180176     NaN       NaN     NaN   63900.0  21201.355469   
2010-02-11   5376.180176     NaN       NaN     NaN   64500.0  21578.271484   
2010-02-12   5271.109863     NaN       NaN     NaN   64700.0  21389.814453   
...                  ...     ...       ...     ...       ...           ...   
2018-08-16  72200.000000  9380.0  161000.0  8250.0  141000.0  53800.000000   
2018-08-17  71700.000000  9410.0  161500.0  8100.0  138500.0  52000.000000   
2018-08-20  71000.000000  9430.0  165500.0  8220.0  137500.0  51100.000000   
2018-08-21  70500.000000  9540.0  165000.0  8280.0  140000.0  52200.000000   
2018-08-22  69100.000000  9570.0  161000.0  8260.0  139000.0  52800.000000   
2018-08-23  67900.000000  9400.0  159000.0  8200.0  136000.0  53000.000000   
2018-08-24  67200.000000  9370.0  159000.0  8220.0  134500.0  52500.000000   
2018-08-27  67200.000000  9520.0  166500.0  8200.0  135000.0  55400.000000   
2018-08-28  67400.000000  9830.0  180000.0  8300.0  135000.0  54600.000000   
2018-08-29  67000.000000  9670.0  176500.0  8290.0  135500.0  54100.000000   
2018-08-30  66000.000000  9520.0  170000.0  8250.0  132500.0  53500.000000   
2018-08-31  67200.000000  9440.0  171500.0  8340.0  132000.0  53000.000000   
2018-09-03  66300.000000  9320.0  175500.0  8310.0  132000.0  52500.000000   
2018-09-04  65300.000000  9230.0  174500.0  8270.0  133500.0  52900.000000   
2018-09-05  66000.000000  9210.0  176000.0  8210.0  132500.0  52900.000000   
2018-09-06  65000.000000  9160.0  172500.0  8170.0  133500.0  52400.000000   
2018-09-07  63500.000000  9320.0  181500.0  8320.0  133500.0  52000.000000   
2018-09-10  61700.000000  9660.0  191

# 집에 돌아와서 다시

In [630]:
def get_multi_index(col_name):
    return pd.MultiIndex.from_product([[col_name],list(raw_data.keys())])
def add_feature(q, col_name, expr):
    ind = get_multi_index(col_name)
    try:
        df = pd.DataFrame(expr.values, columns=ind, index=q.index)
    except AttributeError:
        df = pd.DataFrame(expr, columns=ind, index=q.index)
    q = pd.concat((q, df), axis=1)
    return q

In [624]:
data = {}
for stock, df in raw_data.items():
    dff = df.copy()
    dff.columns = pd.MultiIndex.from_arrays([[stock]*6, df.columns])
    data[stock] = dff

In [627]:
p = pd.DataFrame()
for stock, df in data.items():
    p = pd.concat((p, df), axis=1)

In [638]:
q = p.T.swaplevel().sortlevel().T

In [639]:
import time
start = time.time()
q = add_feature(q, 'pct_change', q.Close.pct_change())
for i in range(3, 31, 2):
    q = add_feature(q, "close_ma_{}".format(i), q.Close.rolling(window=i).mean())
    q = add_feature(q, "volume_ma_{}".format(i), q.Volume.rolling(window=i).mean())
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    q = add_feature(q, "his_vol_{}".format(i), q['pct_change'].rolling(window=i).std()*(252**0.5))
for i in [1, 3, 5, 7, 10, 12, 14, 16]:
    q = add_feature(q, 'returnsClosePrevRaw{}'.format(i), np.log(q.Open / q.Close.shift(1)))
    q = add_feature(q, 'returnsOpenPrevRaw{}'.format(i), np.log(q.Open / q.Open.shift(1)))
for i in [1, 3, 5, 7, 10]:
    q = add_feature(q, 'Y3_{}'.format(i), 
                    np.where(q.Close.rolling(5).mean() - q.Close.shift(1).rolling(10).mean() >=0, 1, 0))
    for shift in [1, 2, 3, 4, 5]:
        q = add_feature(q, 'sh{}_Y3_{}'.format(shift, i), 
                    q['Y3_{}'.format(i)].shift(-1*shift))
elapse = time.time() - start

In [640]:
print(elapse)

28.799176454544067


In [650]:
q2 = q.loc['2013-07-01':'2018-07-01'].copy()

In [651]:
q2

Adj Close                                             \
                   AK홀딩스      BGF    BGF리테일       BNK금융지주        CJ   
Date                                                                  
2013-07-01  26722.099609      NaN       NaN  12409.956055  117000.0   
2013-07-02  27397.300781      NaN       NaN  12705.503906  119500.0   
2013-07-03  27686.699219      NaN       NaN  12578.802734  118000.0   
2013-07-04  27976.199219      NaN       NaN  12832.117188  117000.0   
2013-07-05  28651.400391      NaN       NaN  12958.727539  118000.0   
2013-07-08  28410.300781      NaN       NaN  12958.727539  114000.0   
2013-07-09  28410.300781      NaN       NaN  13296.419922  117500.0   
2013-07-10  27590.300781      NaN       NaN  13169.807617  116500.0   
2013-07-11  29712.599609      NaN       NaN  13254.186523  121000.0   
2013-07-12  30243.199219      NaN       NaN  13085.339844  120000.0   
2013-07-15  31159.699219      NaN       NaN  13000.962891  118000.0   
2013-07-16  30146.699219      NaN       NaN  12916.494141  119500.0   
2013-07-17  30146.699219      NaN       NaN  13043.196289  122500.0   
2013-07-18  31786.699219      NaN       NaN  12621.037109  122500.0   
2013-07-19  31834.900391      NaN       NaN  12747.647461  120000.0   
2013-07-22  32799.601563      NaN       NaN  12621.037109  118500.0   
2013-07-23  32703.199219      NaN       NaN  13085.339844  120500.0   
2013-07-24  33571.398438      NaN       NaN  13085.339844  119000.0   
2013-07-25  33764.300781      NaN       NaN  13127.575195  120000.0   
2013-07-26  35114.898438      NaN       NaN  13254.186523  118500.0   
2013-07-29  35597.199219      NaN       NaN  13169.807617  114000.0   
2013-07-30  37140.800781      NaN       NaN  13465.265625  112500.0   
2013-07-31  38780.699219      NaN       NaN  13549.734375  114500.0   
2013-08-01  38443.101563      NaN       NaN  13338.654297  115000.0   
2013-08-02  37816.000000      NaN       NaN  13423.032227  113500.0   
2013-08-05  39841.898438      NaN       NaN  13338.654297  113500.0   
2013-08-06  39552.500000      NaN       NaN  13127.575195  116000.0   
2013-08-07  39649.000000      NaN       NaN  13169.807617  111000.0   
2013-08-08  39070.101563      NaN       NaN  13043.196289  110000.0   
2013-08-09  37478.398438      NaN       NaN  13000.962891  106500.0   
...                  ...      ...       ...           ...       ...   
2018-05-16  77000.000000  12300.0  186000.0  10500.000000  164000.0   
2018-05-17  75600.000000  12300.0  181500.0  10400.000000  165000.0   
2018-05-18  77700.000000  12350.0  186500.0  10450.000000  166500.0   
2018-05-21  77200.000000  12200.0  185500.0  10350.000000  165500.0   
2018-05-23  76400.000000  12200.0  179000.0  10300.000000  163500.0   
2018-05-24  77900.000000  12150.0  187000.0  10300.000000  160500.0   
2018-05-25  76600.000000  11900.0  185500.0  10250.000000  155000.0   
2018-05-28  78400.000000  11950.0  180000.0   9870.000000  156500.0   
2018-05-29  78000.000000  11700.0  183500.0   9770.000000  154000.0   
2018-05-30  77600.000000  11400.0  177500.0   9430.000000  146000.0   
2018-05-31  80500.000000  11350.0  182500.0   9530.000000  148000.0   
2018-06-01  80700.000000  11450.0  184500.0   9480.000000  146500.0   
2018-06-04  80000.000000  11500.0  188000.0   9790.000000  146500.0   
2018-06-05  80000.000000  11650.0  182500.0   9770.000000  147000.0   
2018-06-07  80600.000000  11900.0  193500.0  10000.000000  153000.0   
2018-06-08  82000.000000  11800.0  196000.0  10000.000000  151000.0   
2018-06-11  84100.000000  11850.0  195500.0  10050.000000  154500.0   
2018-06-12  85900.000000  11700.0  203500.0  10150.000000  154500.0   
2018-06-14  86500.000000  11400.0  199000.0   9790.000000  150500.0   
2018-06-15  83300.000000  11100.0  192500.0   9720.000000  149500.0   
2018-06-18  79200.000000  11100.0  201000.0   9710.000000  146000.0   
2018-06-19  77600.000000  11250.0  209000.0   9640.000000  142500.0   
2018-06-20  79700.000000  11100.0  1960